In [1]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

class TwoPathConv(nn.Module):
    def __init__(self):
        super(TwoPathConv, self).__init__()
        self.upper_layer1 = nn.Sequential(
            nn.Conv2d(4,64,7),
            nn.ReLU(),
            nn.MaxPool2d((4,4),stride = 1)
        )
        self.upper_layer2 = nn.Sequential(
            nn.Conv2d(64,64,3),
            nn.ReLU(),
            nn.MaxPool2d((2,2),stride = 1)
        )
        self.under_layer1 = nn.Sequential(
            nn.Conv2d(4,160,13),
            nn.ReLU()
        )
        self.final_layer = nn.Conv2d(224,5,21)
        
    def forward(self, x):
        upper_x = self.upper_layer2(self.upper_layer1(x))
        under_x = self.under_layer1(x)
        #upper_x = F.max_pool2d(F.relu(self.upper_conv1(x)), (4, 4),stride = 1)
        #upper_x = F.max_pool2d(F.relu(self.upper_conv2(upper_x)), (2, 2), stride = 1)
        #under_x = F.relu(self.under_conv1(x))
        final_x = torch.cat((under_x, upper_x), 1)
        out = self.final_layer(final_x)
        return out
        
net = TwoPathConv()
print(net)

x = Variable(torch.randn(1,4,33,33), requires_grad = True)
y_pred = net.forward(x)
print(y_pred)


TwoPathConv (
  (upper_layer1): Sequential (
    (0): Conv2d(4, 64, kernel_size=(7, 7), stride=(1, 1))
    (1): ReLU ()
    (2): MaxPool2d (size=(4, 4), stride=(1, 1), dilation=(1, 1))
  )
  (upper_layer2): Sequential (
    (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU ()
    (2): MaxPool2d (size=(2, 2), stride=(1, 1), dilation=(1, 1))
  )
  (under_layer1): Sequential (
    (0): Conv2d(4, 160, kernel_size=(13, 13), stride=(1, 1))
    (1): ReLU ()
  )
  (final_layer): Conv2d(224, 5, kernel_size=(21, 21), stride=(1, 1))
)
Variable containing:
(0 ,0 ,.,.) = 
 -0.2936

(0 ,1 ,.,.) = 
 -0.0713

(0 ,2 ,.,.) = 
  0.1108

(0 ,3 ,.,.) = 
  0.5662

(0 ,4 ,.,.) = 
  0.0497
[torch.FloatTensor of size 1x5x1x1]



In [2]:
#get the training set for phase 1
f=open("trainval-balanced.txt", "r")
content=f.readlines()
data=[]
data_train_phase1=[]
i=0
for line in content:
    no_n_line=line[0:len(line)-1]
    item=no_n_line.split(" ")
    data.append([])
    data[i].append(item[0])
    data[i].append(int(item[1]))
    data[i].append(int(item[2]))
    data[i].append(int(item[3]))
    data[i].append(int(item[4]))
    data_train_phase1.append(data[i])
    i += 1
f.close()
print "phase 1 data preparation process completed."

#get the training set for phase 2 and the validation set
data_val=[]
f_in=open("trainval.txt", "r")
content=f_in.readlines()
data=[]
data_val=[]
data_train_phase2=[]
i=0
for line in content:
    no_n_line=line[0:len(line)-1]
    item=no_n_line.split(" ")
    data.append([])
    data[i].append(item[0])
    data[i].append(int(item[1]))
    data[i].append(int(item[2]))
    data[i].append(int(item[3]))
    data[i].append(int(item[4]))
    if i%30000==0:
        data_val.append(data[i])
    else:
        data_train_phase2.append(data[i])
    i += 1
f_in.close()
print "phase 2 data preparation process completed."

import h5py
f = h5py.File('training.h5','r')


phase 1 data preparation process completed.
phase 2 data preparation process completed.


In [3]:
print len(data)
print len(data_val)
print len(data_train_phase1)
print len(data_train_phase2)


NameError: name 'data' is not defined

In [4]:
import h5py
challenge_f = h5py.File('Challenge.h5', 'r') #load challenge data

SAMPLE = [ "HG/0301", "HG/0302",
          "HG/0303", "HG/0304", "HG/0305", "HG/0306", "HG/0307", "HG/0308",
          "HG/0309", "HG/0310", ]
for i in enumerate(SAMPLE):
    index, case = i
    case1 = case[:2]
    case2 = case[3:]
    print(challenge_f[case1][case2].shape)
    
def create_test_batch(img = 0, y = 16, z= 0):
    case = SAMPLE[img]
    case1 = case[:2]
    case2 = case[3:]
    batch = []
    _, X, Y, Z = challenge_f[case1][case2].shape
    for x in range(16, X - 17):
        batch.append(challenge_f[case1][case2][:, x-16:x+17, y-16:y+17, z])
    batch = torch.from_numpy(np.array(batch))
    return batch


(4, 216, 196, 176)
(4, 216, 236, 176)
(4, 216, 236, 176)
(4, 159, 216, 196)
(4, 216, 236, 176)
(4, 162, 230, 230)
(4, 230, 230, 165)
(4, 168, 220, 220)
(4, 162, 230, 230)
(4, 230, 230, 163)
(4, 160, 216, 176)
(4, 160, 216, 176)
(4, 176, 216, 176)
(4, 160, 216, 176)
(4, 160, 216, 176)
(4, 216, 236, 176)
(4, 176, 216, 176)
(4, 160, 216, 176)
(4, 160, 216, 176)
(4, 160, 216, 176)
(4, 176, 216, 176)
(4, 176, 216, 176)
(4, 176, 216, 176)
(4, 176, 216, 176)
(4, 160, 216, 176)
(4, 230, 230, 165)
(4, 168, 240, 240)
(4, 162, 230, 230)
(4, 168, 220, 220)
(4, 162, 230, 230)


In [28]:
target = Variable(torch.LongTensor(np.zeros((1, 128, 128, 96), dtype = int)), requires_grad = False)
target = target.cuda(1)
target = target.view(1, 1,-1)
#print(out)
print(target)
print('fuck')
loss = Variable(torch.Tensor([0]))
for i in range(1):
    #p = (out==i)
    t = (target==i)
    t = Variable(t, dtype = int)
    #pt = torch.mul(p,t)
    print(t)
    print(F.avg_pool1d(t, kernel_size = 1572864) * 1572864)

Variable containing:
(   0   ,.,.) = 
   0   0   0  ...    0   0   0
[torch.cuda.LongTensor of size 1x1x1572864 (GPU 1)]

fuck


TypeError: 'dtype' is an invalid keyword argument for this function

In [14]:
import time
import numpy as np
from torch.autograd import Variable
net = TwoPathConv()
net.load_state_dict(torch.load('premature_net_phase2_without_biasreg.txt'))
net.cuda(3)
pred = {}
prev_time = time.clock()
s = 0
for img in range(1):
    case = SAMPLE[img]
    case1 = case[:2]
    case2 = case[3:]
    _, X, Y, Z = challenge_f[case1][case2].shape
    print(X, Y, Z)
    for y in range(16, Y - 17):
        for z in range(Z):
            s += 1
            if (s%100 == 0):
                print'Ongoing ...' ,(img, y, z)
                print 'time used %.3f' % (time.clock()-prev_time)
            pred[img] = []
            X_batch = create_test_batch(img = img, y = y, z = z)
            X_batch = Variable(X_batch.cuda(3))
            y_pred = net.forward(X_batch)
            y_pred = y_pred.data.cpu().numpy()
            pred[img].append(y_pred.max(axis = 1))
            

(158, 180, 140)
Ongoing ... (0, 16, 0)
time used 0.659
Ongoing ... (0, 16, 1)
time used 0.865
Ongoing ... (0, 16, 2)
time used 1.071
Ongoing ... (0, 16, 3)
time used 1.289
Ongoing ... (0, 16, 4)
time used 1.504
Ongoing ... (0, 16, 5)
time used 1.727
Ongoing ... (0, 16, 6)
time used 1.941
Ongoing ... (0, 16, 7)
time used 2.170
Ongoing ... (0, 16, 8)
time used 2.391
Ongoing ... (0, 16, 9)
time used 2.618
Ongoing ... (0, 16, 10)
time used 2.837
Ongoing ... (0, 16, 11)
time used 3.064
Ongoing ... (0, 16, 12)
time used 3.287
Ongoing ... (0, 16, 13)
time used 3.514
Ongoing ... (0, 16, 14)
time used 3.729
Ongoing ... (0, 16, 15)
time used 3.938
Ongoing ... (0, 16, 16)
time used 4.158
Ongoing ... (0, 16, 17)
time used 4.558
Ongoing ... (0, 16, 18)
time used 4.846
Ongoing ... (0, 16, 19)
time used 5.251
Ongoing ... (0, 16, 20)
time used 5.646
Ongoing ... (0, 16, 21)
time used 6.056
Ongoing ... (0, 16, 22)
time used 6.303
Ongoing ... (0, 16, 23)
time used 6.559
Ongoing ... (0, 16, 24)
time used 

KeyboardInterrupt: 

In [12]:
print torch.cuda.is_available()


True


In [5]:
import torch.nn.init as ninit
def init_net(net):
    for param in net.parameters():
        ninit.uniform(param.data, a=-5e-3, b=5e-3)

In [6]:
def create_batch_phase1(index, batch_size):
    step=len(data_train_phase1)//batch_size
    starting_index=index%step
    X_batch = []
    y_batch = []
    while starting_index < len(data_train_phase1):
        case, x, y, z, l = data_train_phase1[starting_index]
        case1 = case[:2]
        case2 = case[3:]
        X_batch.append(f[case1][case2][:, x-16:x+17, y-16:y+17, z])
        y_batch.append(l)
        starting_index+=step
    X_batch = torch.from_numpy(np.array(X_batch))
    y_batch = torch.from_numpy(np.array(y_batch))
    return X_batch, y_batch

def create_batch_phase2(index, batch_size):
    step=len(data_train_phase2)//batch_size
    starting_index=index%step
    X_batch = []
    y_batch = []
    while starting_index < len(data_train_phase2):
        case, x, y, z, l = data_train_phase2[starting_index]
        case1 = case[:2]
        case2 = case[3:]
        X_batch.append(f[case1][case2][:, x-16:x+17, y-16:y+17, z])
        y_batch.append(l)
        starting_index+=step
    X_batch = torch.from_numpy(np.array(X_batch))
    y_batch = torch.from_numpy(np.array(y_batch))
    return X_batch, y_batch

def create_val(batch_mask):
    X_val=[]
    y_val=[]
    for i in range(len(batch_mask)):
        case, x, y, z, l = data_val[batch_mask[i]]
        case1 = case[:2]
        case2 = case[3:]
        X_val.append(f[case1][case2][:, x-16:x+17, y-16:y+17, z])
        y_val.append(l)
    X_val = torch.from_numpy(np.array(X_val))
    y_val = torch.from_numpy(np.array(y_val))
    return X_val, y_val

In [9]:
import time
import numpy as np
num_train=len(data_train_phase2)
num_val=len(data_val)
batch_size=900
val_size=num_val
num_epoch=2.0
num_times=int(float(num_train)/batch_size*num_epoch)
print num_times
learning_rate = 5e-3
reg=5e-5
net = TwoPathConv()
print(net)
init_net(net)
net.cuda(2)

#create validation set
val_mask=np.random.choice(num_val, val_size)
X_val, y_val = create_val(val_mask)
X_val = Variable(X_val.cuda(2), requires_grad=False)

prev_time = time.clock()
optimizer = torch.optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9, weight_decay=reg)
net.zero_grad()
for i in xrange(num_times):  # loop over the dataset multiple times
    X_batch=None
    X_batch, y_batch = create_batch_phase2(i, batch_size)
    X_batch, y_batch = Variable(X_batch.cuda(2)), Variable(y_batch.cuda(2), requires_grad = False)
    y_pred = net.forward(X_batch)
    y_pred = y_pred.view(-1,5)
    loss = F.cross_entropy(y_pred, y_batch)
    loss.backward()
    optimizer.step()
    net.zero_grad()
    if i % 100 == 0:
        print loss
        print 'time used %.3f' % (time.clock()-prev_time)
        y_val_pred=net.forward(X_val)
        y_val_pred=y_val_pred.view(-1,5)
        useless, predicted=torch.max(y_val_pred.data, 1)
        correct = (predicted == y_val.cuda(2)).sum()
        print('Validation accuracy:', float(correct)/val_size)
print "onephase_net successfully trained!"
torch.save(net.state_dict(), "onephase_net.txt")
print "onephase_net successfully saved!"

55061
TwoPathConv (
  (upper_conv1): Conv2d(4, 64, kernel_size=(7, 7), stride=(1, 1))
  (upper_conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
  (final_layer): Conv2d(64, 5, kernel_size=(21, 21), stride=(1, 1))
)
Variable containing:
 1.6099
[torch.cuda.FloatTensor of size 1 (GPU 2)]

time used 1.986
('Validation accuracy:', 0.7639225181598063)
Variable containing:
 1.3058
[torch.cuda.FloatTensor of size 1 (GPU 2)]

time used 20.896
('Validation accuracy:', 0.8789346246973365)
Variable containing:
 0.7006
[torch.cuda.FloatTensor of size 1 (GPU 2)]

time used 39.665
('Validation accuracy:', 0.8789346246973365)
Variable containing:
 0.4745
[torch.cuda.FloatTensor of size 1 (GPU 2)]

time used 58.660
('Validation accuracy:', 0.8789346246973365)
Variable containing:
 0.4449
[torch.cuda.FloatTensor of size 1 (GPU 2)]

time used 77.659
('Validation accuracy:', 0.8789346246973365)
Variable containing:
 0.4377
[torch.cuda.FloatTensor of size 1 (GPU 2)]

time used 96.598
('Validation 

In [ ]:
def adam(x, dx, config = None):
    if config is None: 
        config = {}
        config.setdefault('learning_rate',1e-4)
        config.setdefault('beta1',0.9)
        config.setdefault('beta2',0.999)
        config.setdefault('epsilon',1e-8)
        config.setdefault('m',torch.FloatTensor(x.size()).zero_().cuda(2))
        config.setdefault('v',torch.FloatTensor(x.size()).zero_().cuda(2))
        config.setdefault('t',0)
    
    next_x = None
    config['t'] += 1
    x.cuda(2)
    dx.cuda(2)
    config['m'] = config['beta1'] * config['m'] + (1 - config['beta1']) * dx
    config['v'] = config['beta2'] * config['v'] + (1 - config['beta2']) * dx**2
    mb = config['m'] / (1 - config['beta1'] ** config['t'])
    vb = config['v'] / (1 - config['beta2'] ** config['t'])
    next_x = x - config['learning_rate'] * mb / (np.sqrt(vb) + config['epsilon'])
    return next_x, config

In [ ]:
def sgd_momentum(w, dw, config = None):

    if config is None: 
        config = {}
        config.setdefault('learning_rate', 1e-2)
        config.setdefault('momentum', 0.9)

    v = config.get('velocity', torch.FloatTensor(w.size()).zero_().cuda(3))
    next_w = None

    v = config['momentum'] * v - config['learning_rate'] * dw
    next_w = w + v
    config['velocity'] = v

    return next_w, config

In [1]:
import pickle as pkl
input1 = open('train_matrix_pred1-21.pkl', 'rb')
matrix_pred = pkl.load(input1)
output1 = open('train_matrix_pred1-21.pkl', 'wb')
pkl.dump(matrix_pred, output1, protocol = 2)

FileNotFoundError: [Errno 2] No such file or directory: 'train_matrix_pred1-21.pkl'